In [2]:
# Dans votre notebook
import pandas as pd

# Charger les features
df = pd.read_parquet('../data/features_engineered.parquet')

# Séparer train et test
train = df[df['TARGET'].notna()].copy()  # Lignes avec TARGET → train
test = df[df['TARGET'].isna()].copy()    # Lignes sans TARGET → test Kaggle

# Préparer pour l'entraînement
X_train = train.drop(['TARGET', 'SK_ID_CURR'], axis=1)
y_train = train['TARGET']

# Pour les prédictions finales
X_test = test.drop(['TARGET', 'SK_ID_CURR'], axis=1)
test_ids = test['SK_ID_CURR']

print(f"Train : {X_train.shape}")
print(f"Test : {X_test.shape}")

Train : (307507, 795)
Test : (48744, 795)


In [3]:
import mlflow
import os

# Configuration MLflow en local (pas Docker)
# Les runs et artifacts seront stockés dans ./mlruns à la racine du projet
tracking_uri = os.path.abspath(os.path.join(os.getcwd(), '..', 'mlruns'))
mlflow.set_tracking_uri(f"file://{tracking_uri}")
mlflow.set_experiment("Projet MLOps - Credit Scoring - Random Forest")

print(f"Tracking URI: {mlflow.get_tracking_uri()}")
print(f"Répertoire de stockage: {tracking_uri}")
print(f"✅ MLflow configuré en local")

# Vérifier la connexion
try:
    experiments = mlflow.search_experiments()
    print(f"Nombre d'expériences: {len(experiments)}")
except Exception as e:
    print(f"⚠️ Erreur: {e}")


Tracking URI: file:///home/zmxw1768/Documents/oc_mlops/mlruns
Répertoire de stockage: /home/zmxw1768/Documents/oc_mlops/mlruns
✅ MLflow configuré en local
Nombre d'expériences: 2


In [4]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
# Séparer avec StratifiedKFold pour garder la même proportion de classes
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=42)

In [5]:
# Vérifier et nettoyer les valeurs problématiques sur TOUS les datasets
import numpy as np

print("🧹 Nettoyage des données...")

# Nettoyer X_train (utilisé pour la CV)
X_train = X_train.replace([np.inf, -np.inf], np.nan).fillna(0)

# Nettoyer X_tr et X_val (utilisés pour les tests simples)
X_tr = X_tr.replace([np.inf, -np.inf], np.nan).fillna(0)
X_val = X_val.replace([np.inf, -np.inf], np.nan).fillna(0)

# Nettoyer X_test (pour les prédictions finales)
X_test = X_test.replace([np.inf, -np.inf], np.nan).fillna(0)

print(f"✅ Données nettoyées")
print(f"   X_train: {X_train.shape}")
print(f"   X_test: {X_test.shape}")


🧹 Nettoyage des données...
✅ Données nettoyées
   X_train: (307507, 795)
   X_test: (48744, 795)


In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (roc_auc_score, recall_score, f1_score, 
                             make_scorer, confusion_matrix)
import numpy as np

# Fermer toute run active
if mlflow.active_run():
    mlflow.end_run()

# Définir le coût métier personnalisé (FN coûte 10x plus que FP)
def business_cost_scorer(y_true, y_pred):
    """
    Coût métier : FN (faux négatif) coûte 10 fois plus cher que FP (faux positif)
    On retourne le négatif du coût pour maximiser (sklearn maximise les scores)
    """
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    cost = fp * 1 + fn * 10  # FN coûte 10x plus
    return -cost  # Négatif car on veut minimiser le coût

# Configuration de la validation croisée
n_splits = 3  # Réduit à 3 pour éviter la surcharge CPU
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Définir les modèles à tester (réduire n_jobs pour éviter surcharge)
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42, n_jobs=1),
    "Random Forest": RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=2, max_depth=10)
}

# Définir les scorers
scoring = {
    'roc_auc': make_scorer(roc_auc_score, needs_proba=True),
    'recall_minority': make_scorer(recall_score, pos_label=1),  # Recall classe 1 (défaut)
    'f1': make_scorer(f1_score, pos_label=1),
    'business_cost': make_scorer(business_cost_scorer)
}

print(f"🔄 Validation croisée avec {n_splits} folds")
print(f"📊 Métriques : ROC-AUC, Recall (classe 1), F1-Score, Coût métier (FN=10x FP)")
print(f"⚙️  Configuration optimisée pour éviter surcharge CPU\n")

# Tester chaque modèle
results = {}
for model_name, model in models.items():
    print(f"🤖 Entraînement : {model_name}")
    
    with mlflow.start_run(run_name=f"{model_name} - CV {n_splits} folds"):
        # Validation croisée avec plusieurs métriques (n_jobs=1 pour éviter surcharge)
        cv_results = cross_validate(
            model, 
            X_train, 
            y_train, 
            cv=skf, 
            scoring=scoring,
            n_jobs=1,  # Séquentiel pour éviter surcharge CPU
            return_train_score=False
        )
        
        # Logger les paramètres
        mlflow.log_param("model", model_name)
        mlflow.log_param("cv_strategy", "StratifiedKFold")
        mlflow.log_param("n_splits", n_splits)
        mlflow.log_param("n_samples", len(X_train))
        mlflow.log_param("n_features", X_train.shape[1])
        mlflow.log_param("business_cost_ratio", "FN=10x FP")
        
        # Logger les paramètres spécifiques du modèle
        for param_name, param_value in model.get_params().items():
            mlflow.log_param(f"model_{param_name}", param_value)
        
        # Stocker et logger les résultats pour chaque métrique
        results[model_name] = {}
        
        for metric_name in scoring.keys():
            scores = cv_results[f'test_{metric_name}']
            mean_score = scores.mean()
            std_score = scores.std()
            
            # Stocker les résultats
            results[model_name][metric_name] = {
                'mean': mean_score,
                'std': std_score,
                'scores': scores
            }
            
            # Logger dans MLflow
            mlflow.log_metric(f"{metric_name}_mean", mean_score)
            mlflow.log_metric(f"{metric_name}_std", std_score)
            
            # Logger les scores de chaque fold
            for i, score in enumerate(scores, 1):
                mlflow.log_metric(f"{metric_name}_fold_{i}", score)
            
            # Affichage formaté
            metric_display = metric_name.replace('_', ' ').title()
            print(f"   {metric_display:20s}: {mean_score:.4f} (±{std_score:.4f})")
        
        # Entraîner le modèle sur toutes les données pour le sauvegarder
        print("   📦 Sauvegarde du modèle...")
        model.fit(X_train, y_train)
        
        # Créer la signature et l'exemple
        if hasattr(model, 'predict_proba'):
            y_pred_proba = model.predict_proba(X_train[:100])[:, 1]
            signature = mlflow.models.signature.infer_signature(X_train, y_pred_proba)
        else:
            signature = mlflow.models.signature.infer_signature(X_train, model.predict(X_train))
        
        input_example = X_train.head(3)
        
        # Logger le modèle
        mlflow.sklearn.log_model(
            model,
            "model",
            signature=signature,
            input_example=input_example
        )
        
        print()

print("\n" + "="*80)
print("📈 RÉSUMÉ DES PERFORMANCES")
print("="*80)
for model_name, metrics in results.items():
    print(f"\n{model_name}:")
    print("-" * 80)
    print(f"{'Métrique':<25s} {'Mean':>12s} {'Std':>12s}")
    print("-" * 80)
    for metric_name, result in metrics.items():
        metric_display = metric_name.replace('_', ' ').title()
        print(f"{metric_display:<25s} {result['mean']:>12.4f} {result['std']:>12.4f}")

print("\n" + "="*80)
print(f"✅ Meilleur AUC-ROC: {max(results, key=lambda k: results[k]['roc_auc']['mean'])}")
print(f"✅ Meilleur Recall: {max(results, key=lambda k: results[k]['recall_minority']['mean'])}")
print(f"✅ Meilleur F1-Score: {max(results, key=lambda k: results[k]['f1']['mean'])}")
print(f"✅ Meilleur Coût Métier: {max(results, key=lambda k: results[k]['business_cost']['mean'])}")
print("="*80)


🔄 Validation croisée avec 3 folds
📊 Métriques : ROC-AUC, Recall (classe 1), F1-Score, Coût métier (FN=10x FP)
⚙️  Configuration optimisée pour éviter surcharge CPU

🤖 Entraînement : Logistic Regression


/home/zmxw1768/Documents/oc_mlops/.venv/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


In [5]:
# S'assurer qu'aucune run n'est active (forcer la fermeture si erreur)
try:
    if mlflow.active_run():
        mlflow.end_run()
except Exception as e:
    print(f"Avertissement lors de la fermeture de la run: {e}")
    # Forcer la désactivation de la run active en mémoire
    import mlflow.tracking.fluent
    mlflow.tracking.fluent._active_run_stack = []

# Utiliser un context manager pour gérer automatiquement la run
with mlflow.start_run(run_name="Random Forest Baseline"):
    # Entraîner le modèle
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X_tr, y_tr)
    
    # Prédire sur la validation
    y_pred_proba = rf.predict_proba(X_val)[:, 1]
    
    # Calculer le ROC AUC
    auc = roc_auc_score(y_val, y_pred_proba)
    print(f"Validation ROC AUC: {auc:.4f}")
    
    # Log des paramètres
    mlflow.log_param("model", "RandomForest")
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("random_state", 42)
    mlflow.log_param("train_size", len(X_tr))
    mlflow.log_param("val_size", len(X_val))
    
    # Log des métriques
    mlflow.log_metric("roc_auc", auc)
    
    # Créer la signature et un exemple d'entrée
    from mlflow.models.signature import infer_signature
    signature = infer_signature(X_tr, y_pred_proba)
    input_example = X_tr.head(3)
    
    # Logger le modèle localement avec signature et exemple
    mlflow.sklearn.log_model(
        rf,
        "model",
        signature=signature,
        input_example=input_example
    )
    
    print(f"✅ Modèle sauvegardé dans ./mlruns avec signature!")
    print(f"📊 Taille d'entrée: {X_tr.shape}")
    
    # Le context manager ferme automatiquement la run à la fin du bloc

Validation ROC AUC: 0.7127


2026/02/09 16:21:41 WARNING mlflow.models.signature: Failed to infer schema for inputs. Setting schema to `Schema([ColSpec(type=AnyType())]` as default. Note that MLflow doesn't validate data types during inference for AnyType. To see the full traceback, set logging level to DEBUG.
2026/02/09 16:21:45 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2026/02/09 16:21:45 WARNING mlflow.models.model: Failed to validate serving input example {
  "dataframe_split": {
    "columns": [
      "CODE_GENDER",
      "FLAG_OWN_CAR",
      "FLAG_OWN_REALTY",
      "CNT_CHILDREN",
      "AMT_INCOME_TOTAL",
      "AMT_CREDIT",
      "AMT_ANNUITY",
      "AMT_GOODS_PRICE",
      "REGION_POPULATION_RELATIVE",
      "DAYS_BIRTH",
      "DAYS_EMPLOYED",
      "DAYS_REGISTRATION",
      "DAYS_ID_PUBLISH",
      "OWN_CAR_AGE",
      "FLAG_MOBIL",
      "FLAG_EMP_PHONE",
      "FLAG_WORK_PHONE",
     

✅ Modèle sauvegardé dans ./mlruns avec signature!
📊 Taille d'entrée: (246005, 795)


## Interface MLflow

Pour visualiser vos expériences MLflow, lancez l'interface web depuis votre environnement uv:

**Option 1 - Depuis le terminal VSCode:**
```bash
cd /home/zmxw1768/Documents/oc_mlops
uv run mlflow ui
```

**Option 2 - Via l'environnement virtuel:**
```bash
source .venv/bin/activate
mlflow ui
```

Ensuite, ouvrez votre navigateur sur: **http://localhost:5000**

> 💡 Les données sont stockées localement dans le dossier `./mlruns` (paramètres, métriques et artifacts)